# IceVision Deployment App Example
[IceVision](https://github.com/airctic/IceVision) features:

✔ Data curation/cleaning with auto-fix 

✔ Exploratory data analysis dashboard 

✔ Pluggable transforms for better model generalization  

✔ Access to hundreds of neural net models (Torchvision, MMDetection, EfficientDet, Timm)

✔ Access to multiple training loop libraries (Pytorch-Lightning, Fastai)

✔ Multi-task training to efficiently combine object 
detection, segmentation, and classification models  

## Installing packages

In [ ]:
!wget https://raw.githubusercontent.com/airctic/icevision/master/install_icevision_inference.sh
!bash install_icevision_inference.sh colab

In [ ]:
!echo "- Installing gradio"
!pip install gradio -U -q

In [ ]:
# Restart kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Imports

In [ ]:
from icevision.all import *
import icedata
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

## Loading trained model

In [2]:
class_map = icedata.coco.class_map()

In [ ]:
# Loading model from IceZoo (IceVision Hub)
model = icedata.coco.trained_models.faster_rcnn_resnet50_fpn()

In [4]:
# Transforms
image_size = 384
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])

## Defining the `show_preds` method: called by `gr.Interface(fn=show_preds, ...)`

In [5]:
# Setting the model type: used in end2end_detect() method here below
model_type = models.torchvision.faster_rcnn

In [13]:
def show_preds(input_image, display_label, display_bbox, detection_threshold):

    if detection_threshold==0: detection_threshold=0.5

    img = PIL.Image.fromarray(input_image, 'RGB')

    pred_dict  = model_type.end2end_detect(img, valid_tfms, model, class_map=class_map, detection_threshold=detection_threshold,
                                           display_label=display_label, display_bbox=display_bbox, return_img=True, 
                                           font_size=16, label_color="#FF59D6")

    return pred_dict['img']

## Gradio User Interface
You have 3 options to feed the model (by commenting/uncommenting the code here below):

1- Load an image from local drive

2- Grab an image from the webcam

3- Continuous image stream from the webcam

> Note: You might check the Label and the BBox checkboxes, and set the Detection Thereshold to another value

In [ ]:
# display_chkbox = gr.inputs.CheckboxGroup(["Label", "BBox"], label="Display", default=True)
display_chkbox_label = gr.inputs.Checkbox(label="Label", default=True)
display_chkbox_box = gr.inputs.Checkbox(label="Box", default=True)

detection_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Detection Threshold")

outputs = gr.outputs.Image(type="pil")

# Option 1: Get an image from local drive
gr_interface = gr.Interface(fn=show_preds, inputs=["image", display_chkbox_label, display_chkbox_box,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO')

# #  Option 2: Grab an image from a webcam
# gr_interface = gr.Interface(fn=show_preds, inputs=["webcam", display_chkbox_label, display_chkbox_box,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO', live=False)

# #  Option 3: Continuous image stream from the webcam
# gr_interface = gr.Interface(fn=show_preds, inputs=["webcam", display_chkbox_label, display_chkbox_box,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO', live=True)


gr_interface.launch(inline=False, share=True, debug=True)
